This is the original page of bj aq data preprocess. Functions based on this page can be found [here](https://github.com/txytju/air-quality-prediction/blob/master/preprocess/aq_data_preprocess.py)

In [5]:
# Using pandas to process data
from collections import Counter
import numpy as np
import pandas as pd
import datetime
from matplotlib import pyplot as plt
%matplotlib inline
from scipy.stats import pearsonr

from utils.data_util import load_bj_aq_data

%load_ext autoreload
%autoreload 2

### 1. 数据载入

In [6]:
bj_aq_data, stations, bj_aq_stations, bj_aq_stations_merged = load_bj_aq_data()

In [7]:
print("最早的日期：", bj_aq_stations_merged.index.min())
print("最晚的日期：", bj_aq_stations_merged.index.max())

最早的日期： 2017-01-01 14:00:00
最晚的日期： 2018-04-23 23:00:00


In [8]:
bj_aq_stations_merged.shape

(10823, 211)

### 2. 重复日期的值除去

- 虽然数据中存在着很多的缺失值，但是在一些时间点上数据是重复的，因此首先去除重复的时间点。
- 思路是：
    1. 首先让数字成为 df index
    2. 遍历 index ，找出对应的时间的重复值，删除后出现的重复时间

In [9]:
df_merged = bj_aq_stations_merged

In [10]:
df_merged["time"] = df_merged.index

In [11]:
df_merged.set_index("order", inplace=True)

In [12]:
print("重复值去除之前，共有数据数量", df_merged.shape[0])

重复值去除之前，共有数据数量 10823


In [13]:
used_times = []
for index in df_merged.index :
    time = df_merged.loc[index]["time"]
    if time not in used_times :
        used_times.append(time)
    else : 
        df_merged.drop([index], inplace=True)

In [14]:
print("重复值去除之后，共有数据数量", df_merged.shape[0])

重复值去除之后，共有数据数量 10629


In [15]:
df_merged.set_index("time", inplace=True)

### 3. 缺失值的分析

- 缺失值分析包括
    - 整小时的缺失：哪些时间节点上，所有站点的所有特征都缺失了
    - 某个小时的某个站点的所有数据缺失
    - 某个小时的某个站点的某个数据缺失
- 对于上述第一种情况，如果某天的缺失数据超过5个小时，就放弃使用该天的数据，如果没有超过5个小时，使用插值的方式对数据进行填充。
- 对于后两种情况，使用距离该站的有数据的最近一个站的数据，直接作为该站的数据。

In [16]:
min_time = df_merged.index.min()
max_time = df_merged.index.max()

min_time = datetime.datetime.strptime(min_time, '%Y-%m-%d %H:%M:%S')
max_time = datetime.datetime.strptime(max_time, '%Y-%m-%d %H:%M:%S')
delta_all = max_time - min_time
print("在空气质量数据时间段内，总共应该有 %d 个小时节点。" %(delta_all.total_seconds()/3600 + 1))
print("实际的时间节点数是 %d" %(df_merged.shape[0]))
print("缺失时间节点数量是 %d" %(10898-10113))

在空气质量数据时间段内，总共应该有 11458 个小时节点。
实际的时间节点数是 10629
缺失时间节点数量是 785


#### 3.1 整小时缺失
统计哪些时间节点发生了整小时缺失的情况

In [17]:
delta = datetime.timedelta(hours=1)
time = min_time
missing_hours = []
missing_hours_str = []

In [18]:
while time <=  max_time :
    if datetime.date.strftime(time, '%Y-%m-%d %H:%M:%S') not in df_merged.index :
        missing_hours.append(time)
        missing_hours_str.append(datetime.date.strftime(time, '%Y-%m-%d %H:%M:%S'))
    time += delta

In [19]:
print("整小时的缺失共计 : ", len(missing_hours))

整小时的缺失共计 :  829


#### 3.2 某个小时某个站点数据缺失
- 在没有全部缺失的小时处，某个站点会在某个小时出现全部数据缺失的情况。这种情况下，使用距离该站最近的站的数据对其进行补全。
- 或者某个站点的某个值缺失，此时使用相邻站点的数据补全

In [20]:
from utils.weather_data_util import get_station_locations, get_location_lists

In [21]:
aq_station_locations = pd.read_excel("./KDD_CUP_2018/Beijing/location/Beijing_AirQuality_Stations_locations.xlsx", sheet_name=1)

In [22]:
aq_station_locations.head()

,stationName,longitude,latitude
0,dongsi_aq,116.417,39.929
1,tiantan_aq,116.407,39.886
2,guanyuan_aq,116.339,39.929
3,wanshouxigong_aq,116.352,39.878
4,aotizhongxin_aq,116.397,39.982


对于一个空气质量站点，将其他站点按照距该站点距离的大小关系排列，并保存成列表

In [23]:
for index_t in aq_station_locations.index:
    row_t = aq_station_locations.loc[index_t]
    # location of target station
    long_t = row_t["longitude"]
    lati_t = row_t["latitude"]
    # column name
    station_name = row_t["stationName"]
    
    # add a new column to df
    all_dis = []
    for index in aq_station_locations.index:
        row = aq_station_locations.loc[index]
        long = row['longitude']
        lati = row['latitude']
        dis = np.sqrt((long-long_t)**2 + (lati-lati_t)**2)
        all_dis.append(dis)
    
    aq_station_locations[station_name] = all_dis

In [24]:
# 不同站之间的距离关系
aq_station_locations

,stationName,longitude,latitude,dongsi_aq,tiantan_aq,guanyuan_aq,wanshouxigong_aq,aotizhongxin_aq,nongzhanguan_aq,wanliu_aq,...,miyunshuiku_aq,donggaocun_aq,yongledian_aq,yufa_aq,liulihe_aq,qianmen_aq,yongdingmennei_aq,xizhimenbei_aq,nansanhuan_aq,dongsihuan_aq
0,dongsi_aq,116.417,39.929,0.000000,0.044147,0.078000,0.082620,0.056648,0.044721,0.142352,...,0.754278,0.723498,0.425494,0.425406,0.543774,0.037202,0.057775,0.072450,0.087920,0.066753
1,tiantan_aq,116.407,39.886,0.044147,0.000000,0.080455,0.055579,0.096519,0.074277,0.156847,...,0.793590,0.744423,0.414309,0.381320,0.509200,0.017692,0.016401,0.089376,0.049204,0.092655
2,guanyuan_aq,116.339,39.929,0.078000,0.080455,0.000000,0.052631,0.078568,0.122262,0.077897,...,0.807517,0.799501,0.494191,0.410855,0.486541,0.063530,0.076381,0.026926,0.078549,0.144347
3,wanshouxigong_aq,116.352,39.878,0.082620,0.055579,0.052631,0.000000,0.113318,0.123944,0.126909,...,0.835537,0.799442,0.461863,0.361757,0.461203,0.047854,0.042048,0.076059,0.027203,0.144506
4,aotizhongxin_aq,116.397,39.982,0.056648,0.096519,0.078568,0.113318,0.000000,0.078237,0.110114,...,0.729030,0.732566,0.471058,0.472073,0.564989,0.083024,0.106042,0.055570,0.129294,0.096151
5,nongzhanguan_aq,116.461,39.937,0.044721,0.074277,0.122262,0.123944,0.078237,0.000000,0.181041,...,0.719961,0.678859,0.392822,0.447001,0.583069,0.076158,0.090609,0.113283,0.123329,0.022091
6,wanliu_aq,116.287,39.987,0.142352,0.156847,0.077897,0.126909,0.110114,0.181041,0.000000,...,0.807168,0.840630,0.567134,0.467181,0.498014,0.139313,0.154175,0.070235,0.154019,0.201792
7,beibuxinqu_aq,116.174,40.090,0.291496,0.309685,0.230534,0.276818,0.247776,0.325235,0.152899,...,0.842882,0.946053,0.716774,0.583760,0.538865,0.292099,0.306914,0.221633,0.303961,0.343922
8,zhiwuyuan_aq,116.207,40.002,0.222326,0.231206,0.150841,0.190790,0.191050,0.262185,0.081394,...,0.861757,0.918245,0.644884,0.490890,0.470035,0.214367,0.225488,0.149893,0.217341,0.283099
9,fengtaihuayuan_aq,116.279,39.863,0.152971,0.130050,0.089196,0.074525,0.167586,0.196469,0.124258,...,0.896616,0.873756,0.526134,0.343642,0.397404,0.121458,0.115732,0.114809,0.089275,0.217697


In [25]:
# 以每一个站的名字为 key，以其他站的名字组成的列表为 value list，列表中从前向后距离越来越远
near_stations = {}
for index_t in aq_station_locations.index:
    target_station_name = aq_station_locations.loc[index_t]['stationName']
    ordered_stations_names = aq_station_locations.sort_values(by=target_station_name)['stationName'].values[1:]
    near_stations[target_station_name] = ordered_stations_names

In [26]:
# 举个例子：dingling_aq 附近的、按照距离排序的站的名字
near_stations['dingling_aq']

array(['pingchang_aq', 'beibuxinqu_aq', 'badaling_aq', 'zhiwuyuan_aq',
       'yanqin_aq', 'wanliu_aq', 'aotizhongxin_aq', 'xizhimenbei_aq',
       'mentougou_aq', 'gucheng_aq', 'guanyuan_aq', 'huairou_aq',
       'dongsi_aq', 'nongzhanguan_aq', 'qianmen_aq', 'fengtaihuayuan_aq',
       'wanshouxigong_aq', 'dongsihuan_aq', 'tiantan_aq',
       'yongdingmennei_aq', 'nansanhuan_aq', 'shunyi_aq', 'yungang_aq',
       'fangshan_aq', 'yizhuang_aq', 'tongzhou_aq', 'daxing_aq',
       'miyun_aq', 'miyunshuiku_aq', 'liulihe_aq', 'yufa_aq',
       'yongledian_aq', 'pinggu_aq', 'donggaocun_aq'], dtype=object)

#### 3.3 个别缺失的处理

In [27]:
def get_estimated_value(station_name, feature_name, near_stations, row):
    '''
    为 feature 寻找合理的缺失值的替代。
    Args:
        near_stations : a dict of {station : near stations}
    '''   
    near_stations = near_stations[station_name]    # A list of nearest stations
    for station in near_stations :                 # 在最近的站中依次寻找非缺失值
        feature = station + "_" +feature_name
        if not pd.isnull(row[feature]):
            return row[feature]
        return 0

In [28]:
for index in df_merged.index :
    row = df_merged.loc[index]
    for feature in row.index :
        # print(feature)
        if pd.isnull(row[feature]) :
            elements = feature.split("_")                  # feature example： nansanhuan_aq_PM2.5
            station_name = elements[0] + "_" + elements[1] # nansanhuan_aq
            feature_name = elements[2]                     # PM2.5
            row[feature] = get_estimated_value(station_name, feature_name, near_stations, row)

In [29]:
# 现在数据中没有缺失值了 :)
pd.isnull(df_merged).any().any()

False

In [30]:
df_merged.shape

(10629, 210)

#### 3.4 整小时的缺失的处理

统计缺失小时的连续值
- 如果一个缺失小时在一个长度小于等于5小时的缺失时段内，就进行补全
- 如果超过5小时，就舍弃，将全部值补成 NAN，**这也是整个表中唯一可能出现 NAN 的情况**

In [31]:
# 将 missing hours 分成两类 : keep_hours and drop_hours
keep_hours = []
drop_hours = []

先对小于5小时的进行填充

In [32]:
delta = datetime.timedelta(hours=1)

for hour in missing_hours_str : 
    
    time = datetime.datetime.strptime(hour, '%Y-%m-%d %H:%M:%S')
    
    # 前边第几个是非空的
    found_for = False
    i = 0
    while not found_for :
        i += 1
        for_time = time - i * delta
        for_time_str = datetime.date.strftime(for_time, '%Y-%m-%d %H:%M:%S')
        if for_time_str in df_merged.index :
            for_row = df_merged.loc[for_time_str]
            for_step = i
            found_for = True
            
            
    # 后边第几个是非空的
    found_back = False
    j = 0
    while not found_back :
        j += 1
        back_time = time + j * delta
        back_time_str = datetime.date.strftime(back_time, '%Y-%m-%d %H:%M:%S')
        if back_time_str in df_merged.index :
            back_row = df_merged.loc[back_time_str]
            back_step = j
            found_back = True
    
    # print(for_step, back_step)
    all_steps = for_step + back_step
    if all_steps > 5 :
        drop_hours.append(hour)
    else : 
        keep_hours.append(hour)
        # 插值
        delata_values = back_row - for_row
        df_merged.loc[hour] = for_row + (for_step/all_steps) * delata_values        

In [33]:
# keep 180 hours of 785 missing hours
print(len(drop_hours), len(keep_hours), len(missing_hours_str))

605 224 829


In [34]:
print(df_merged.shape)

(10853, 210)


In [35]:
# 依然 没有 Nan，棒！
pd.isnull(df_merged).any().any()

False

再对超过5小时的填充 NAN

In [36]:
nan_series = pd.Series({key:np.nan for key in df_merged.columns})

for hour in drop_hours:
    df_merged.loc[hour] = nan_series

In [37]:
df_merged.sort_index(inplace=True)

In [38]:
# 11458 和应有的数量一致 :)
df_merged.shape

(11458, 210)

In [39]:
df_merged.to_csv("test/bj_aq_data.csv")

### 4. 数据归一化

In [40]:
describe = df_merged.describe()
describe.to_csv("data/bj_aq_describe.csv")

In [41]:
df_norm = (df_merged - describe.loc['mean']) / describe.loc['std']
df_norm.to_csv("data/bj_aq_norm_data.csv")

# ChangeLog
- 0424
    - 对原空气质量数据进行了处理，包括：
        - 删去重复日期
        - 缺失值分析
            - 整体缺失数据(指所有站点的所有特征在这一时刻均无数据)
                - 缺失不超过5小时：使用前后线性插值的方式
                    - 暂时没有考虑 “基于天的周期性”
                - 缺失超过5小时：放弃该时刻的值，使用 NAN 替代
            - 局部缺失数据(指某些站点的某些特征在这一刻没有数据)
                - 解决方法：收集距离该站最近的且该特征有数值的特征，并填充该值
    - 下载新数据，并进行数据融合
- 0425
    - 实现了对数据的归一化
- 0605
    - 整理项目内容